# Ukol 01 - Build GLM model to predict attrition of employee (team work)
* split data - test, train, validation (should be done)
* train GLM (logistic regression) model to predict attrition of given employe on training set (should be done)
* cross validate on test sample 
* find the 'best' model based on precision (should be done)
* compute precision on validate sample (should be done)
* paste your result into Slack - lecture_05
* you can paste working results too to made other teams to work hard

In [42]:
import numpy as np
import pandas as pd  # library when dealing with tabular data
import os  # standard set of OS-tools
import sys

import psycopg2        # for connection to PostgreSQL DB
import psycopg2.extras # for Dictionary-like cursor

from matplotlib import pyplot
import matplotlib.pyplot as plt

In [43]:
print(os.getcwd())

C:\Adam\School_FJFI\magisterske\3\DAS\DAS-ukoly\ukol_01


In [44]:
empData = pd.read_csv("empdata.csv")

In [45]:
empData.head(5)

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_count,employee_number,...,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_curr_manager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [46]:
print(empData.shape)

(1470, 35)


In [47]:
print(empData.columns)

Index(['age', 'attrition', 'business_travel', 'daily_rate', 'department',
       'distance_from_home', 'education', 'education_field', 'employee_count',
       'employee_number', 'environment_satisfaction', 'gender', 'hourly_rate',
       'job_involvement', 'job_level', 'job_role', 'job_satisfaction',
       'marital_status', 'monthly_income', 'monthly_rate',
       'num_companies_worked', 'over18', 'over_time', 'percent_salary_hike',
       'performance_rating', 'relationship_satisfaction', 'standard_hours',
       'stock_option_level', 'total_working_years', 'training_times_last_year',
       'work_life_balance', 'years_at_company', 'years_in_current_role',
       'years_since_last_promotion', 'years_with_curr_manager'],
      dtype='object')


In [48]:
empData['attrition'] = empData['attrition'].map({'Yes': 1, 'No': 0})

In [55]:
from sklearn.model_selection import train_test_split
shuffled = empData.sample(frac=1, random_state=42)
#split dataset to train and test
data_train, data_test = train_test_split(shuffled, test_size=0.2, random_state=42)

In [56]:
#split train test further to train and val
data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)

In [57]:
X_train = data_train.loc[:, empData.columns != 'attrition']
y_train = data_train['attrition']
X_val = data_val.loc[:, empData.columns != 'attrition']
y_val = data_val['attrition']
X_test = data_test.loc[:, empData.columns != 'attrition']
y_test = data_test['attrition']

In [60]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(1058, 34)
(118, 34)
(294, 34)


In [62]:
#I tried logistic regression in R and these variables seemed good (but over 10 were stat. significant)
features = ['environment_satisfaction','job_involvement','job_level', 'job_satisfaction']

## Logistic regression

In [105]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

In [133]:
penalties = ['l1', 'l2']
acc_val = [0] * len(penalties)
precision = [0] * len(penalties)
for idx, regularizer in enumerate(list(penalties)):
    lgr = LogisticRegression(penalty = regularizer)
    lgr.fit(X_train[features], y_train)
    
    y_val_pred = pd.Series(lgr.predict(X_val[features]), name = 'prediction')
    acc_val[idx] = accuracy_score(y_val_pred, y_val)
    precision[idx] = precision_recall_fscore_support(y_val_pred, y_val)[0]
    print('Regularizer:', regularizer, ', accuracy: {:.10f}'.format(acc_val[idx]), 'Precision:', precision[idx])

Regularizer: l1 , accuracy: 0.8559322034 Precision: [1.         0.05555556]
Regularizer: l2 , accuracy: 0.8559322034 Precision: [1.         0.05555556]


c:\users\admin\.conda\envs\test_venv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\.conda\envs\test_venv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [134]:
acc_val

[0.8559322033898306, 0.8559322033898306]

In [135]:
precision_recall_fscore_support(y_val_pred, y_val)[0]

array([1.        , 0.05555556])

In [136]:
#kfold = cross_val_score(lgr, X_train[features], y_train, cv=2)
#print('5-fold cross-validation results:' , kfold)

### Final evaluation

In [137]:
y_test_pred = pd.Series(lgr.predict(X_test[features]), name = 'prediction')
acc_test = accuracy_score(y_test_pred, y_test)
print("Accuracy is:", acc_test)

Accuracy is: 0.8367346938775511


In [138]:
cm = confusion_matrix(y_test_pred, y_test)
print(cm)

[[245  48]
 [  0   1]]


In [140]:
precision_recall_fscore_support(y_test_pred, y_test)[0]

array([1.        , 0.02040816])

## Inserting into database